# **Deploy Gemma fine-tuning model and Test after Fine-tuning for Receipt Extraction**

In [ ]:
!pip install datasets transformers accelerate peft trl bitsandbytes python-dotenv packaging

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 183.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 208.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 196.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 228.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 42.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 104.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 232.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 191.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 47.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 89.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 71.4 MB/s eta 0:0

In [ ]:
# Import library after install all libraries
import os
import torch
import json
from transformers import pipeline
from peft import LoraConfig

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
# Load environment and save HuggingFace token
from dotenv import load_dotenv
load_dotenv("env.txt")
token = os.getenv("HUGGINGFACE_TOKEN")
!huggingface-cli login --token {token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `maujadiaiengineer` has been saved to /home/ec2-user/.cache/huggingface/stored_tokens
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful.
The current active token is: `maujadiaiengineer`


In [ ]:
# Create inference pipeline based my Gemma fine-tuned model
pipeline = pipeline(
    task="image-text-to-text", # required for image input
    model="budionosan/gemma-finetuning-receipts-extraction",
    device="cuda",
    torch_dtype=torch.bfloat16
)

adapter_config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
# Create function of receipt extraction
def receipt_extraction(image):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": "Extract store name, date and total information based this receipt photo"},
            ]
        },
    ]

    response = pipeline(text=messages, max_new_tokens=128, return_full_text=False, temperature=1)
    result = response[0]['generated_text']
    return result

In [ ]:
receipt_extraction("gokbbq.jpg")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0629 08:30:09.958000 8404 site-packages/torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


"['GO! K BBQ', '24-12-2023', '$92.66']"

In [ ]:
receipt_extraction("ovenfc.jpg")

"['OVEN & FRIED CHICKEN', '09-02-2024', '$51.55']"

In [ ]:
receipt_extraction("strongflour.jpg")

"['STRONG FLOUR', '27-05-2018', '$77.05']"